In [ ]:
import sys
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from PySide6.QtWidgets import QApplication, QWidget, QVBoxLayout, QLabel, QLineEdit, QPushButton, QTextEdit
from PySide6.QtCore import QCoreApplication
import datetime

# Check if a QApplication already exists; create it if not
app = QCoreApplication.instance()
if app is None:
    app = QApplication(sys.argv)

class ArticleSearcherWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Article Searcher")
        self.resize(800, 600)  # Larger window size

        # Layout
        self.layout = QVBoxLayout(self)

        # Keyword input
        self.keyword_label = QLabel("Enter Keyword:")
        self.keyword_input = QLineEdit()
        self.layout.addWidget(self.keyword_label)
        self.layout.addWidget(self.keyword_input)

        # Buttons for Websites
        self.benzinga = QPushButton("Benzinga", self)
        self.layout.addWidget(self.benzinga)
        self.benzinga.setCheckable(True)
        
        self.marketbeat = QPushButton("MarketBeat", self)
        self.layout.addWidget(self.marketbeat)
        self.marketbeat.setCheckable(True)
        
        self.yahoo = QPushButton("Yahoo Finance", self)
        self.layout.addWidget(self.yahoo)
        self.yahoo.setCheckable(True)
        
        self.dailyfx = QPushButton("DailyFX", self)
        self.layout.addWidget(self.dailyfx)
        self.dailyfx.setCheckable(True)
        
        self.seeking = QPushButton("Seeking Alpha", self)
        self.layout.addWidget(self.seeking)
        self.seeking.setCheckable(True)
                
        # Search button
        self.search_button = QPushButton("SEARCH")
        self.search_button.clicked.connect(self.fetch_articles)
        self.layout.addWidget(self.search_button)
        self.search_button.setStyleSheet("background-color : rgb(87, 207, 99)")
        
        # Display area
        self.result_area = QTextEdit()
        self.result_area.setReadOnly(True)
        self.layout.addWidget(self.result_area)

    def fetch_articles(self):
        keyword = self.keyword_input.text()
        self.result_area.clear()
        self.result_area.append(f"Searching for articles related to: '{keyword}'\n")
        articles = self.search_articles(keyword)
        self.display_articles(articles)
    
    def search_articles(self, keyword):
        urls = []
        if self.benzinga.isChecked():
            urls.append('https://www.benzinga.com')
        if self.marketbeat.isChecked():
            urls.append('https://www.marketbeat.com')
        if self.yahoo.isChecked():
            urls.append('https://finance.yahoo.com')
        if self.dailyfx.isChecked():
            urls.append('https://www.dailyfx.com')
        if self.seeking.isChecked():
            urls.append('https://www.seekingalpha.com')

        
        all_links = []
        for url in urls:
            links = self.process_url(url, keyword)
            all_links.append((url, links))
        return all_links

    def process_url(self, url, keyword):
        self.result_area.append(f"Processing {url}...")
        soup = self.get_soup(url)
        if soup:
            related_links = self.extract_related_articles(soup, url, keyword)
            return related_links
        return []

    def get_soup(self, url):
        headers = {
            'User-Agent': 'Mozilla/5.0'
        }
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            return BeautifulSoup(response.text, 'html.parser')
        except Exception as e:
            self.result_area.append(f"Error fetching {url}: {str(e)}")
            return None

    def extract_related_articles(self, soup, base_url, keyword):
        links = []
        if not soup:
            return links
        for element in soup.find_all('a', href=True):
            text = element.get_text(" ", strip=True)
            if keyword.lower() in text.lower():
                link = urljoin(base_url, element['href'])
                links.append(link)
        return links

    def display_articles(self, articles):
        now = datetime.datetime.now()
        date_time_str = now.strftime("%Y-%m-%d %H:%M:%S")

        with open('search_results.txt', 'a') as file:
            search_header = f"Search Term: '{self.keyword_input.text()}' on {date_time_str}\n"
            self.result_area.append(search_header)
            file.write(search_header)

            if articles:
                for url, links in articles:
                    if links:
                        display_text = f"Found {len(links)} articles in {url}:\n"
                        self.result_area.append(display_text)
                        file.write(display_text)
                        for link in links:
                            self.result_area.append(link)
                            file.write(link + '\n')
                    else:
                        no_articles_text = f"No related articles found in {url}\n"
                        self.result_area.append(no_articles_text)
                        file.write(no_articles_text)
                    separator = "-" * 80 + '\n'
                    self.result_area.append(separator)
                    file.write(separator)
            else:
                no_articles_text = "No articles found for any URLs.\n"
                self.result_area.append(no_articles_text)
                file.write(no_articles_text)

            file.write('\n\n')


# GUI Application Setup
window = ArticleSearcherWindow()
window.show()
app.exec()
